# Data Visualization & Business Analysis

### About today

- We will be using the Superstore dataset, consisting of 9800 data points and 18 features
- We will try to answer several questions regarding of how the business has performed.

The business would probably have certain metrics to measure the business performance (e.g.: the northstar metric or OKR).

We will not be focusing on postulating the next year's OKR but it probably would go as something like:
    - Objective 1: Be the go to 
    Key results:
    1. Increase sales number by xx %
    2. Increase transaction number by xx %
    
    - Objective 2: Be the most trusted one stop superstore in the country
    Key results:
    1. Increase customer retention number by xx %
    2. Increase foot traffic by xx %
    3. Increase shipping capacity by xx %
    
Assuming that we get the objectives and key results right, we can see that the business might want to focus on:
1. Sales pattern
2. Transaction pattern
3. Customer behavior (i.e.: retention)
4. Shipping schedule traffic


Those four are going to be our main theme for today.

The goal for this course is to help the business to understand the numbers in the right context and help them in making decisions.

# Import package

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

sns.set_style("darkgrid")
sns.set(font_scale = 1.3)

%matplotlib inline

# Import Data

In [ ]:
df = pd.read_csv("data/Superstore Sales Dataset.csv", parse_dates=['Order Date', 'Ship Date'])
print("Row number: {:d}".format(df.shape[0]))
print("Column number: {:d}".format(df.shape[1]))
df.head()

# Foundational Exploration

Few questions to ask:
1. What's the transaction looks like for each segment, region, and category?
2. What's the sales number look like for each category?
3. Each region?
4. What's the total sales each year?
5. What's the average sales each year?

## Sales

### Sales points by category

In [ ]:
sns.catplot(data=df,
           x='Category',
            y='Sales',
            # change it to boxplot
            kind='strip',
            # hide outliers

            height=8,
            aspect=1
           )

# change y-scale to log

plt.show()

- For every sale point, it gets represented by a dot.
- We can clearly see that the range of sale numbers for furniture and technology are much higher than office supplies

In [ ]:
# Bonus: try ecdf

# change x-scale to log

### Total sales by category

In [ ]:
df_sales = df.groupby("Category").agg({'Order ID':'count',
                              'Sales': 'sum'
                              })


df_sales.rename(columns={'Order ID':'Count'}, inplace=True)

df_sales.reset_index(inplace=True)

print(df_sales.shape)
df_sales.head()

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))

sns.barplot(data=df_sales,
           x='Category',
            y='Sales'
           )

plt.title("Total Sales by Category")

plt.show()

### Sales distribution by segment and region

In [ ]:
sns.catplot(data=df,
           x='Category',
            y='Sales',
            col='Region',
            row='Segment',
            kind='box',
            showfliers=False,
            height=8,
            aspect=0.5
           )
plt.show()

### Annual Sales Sum

In [ ]:
df_sales = df[['Order Date', 'Sales']].copy()
df_sales.set_index('Order Date', inplace=True)
df_sales.head()

In [ ]:
df_sales_year = df_sales.resample('Y').sum()

fig, ax = plt.subplots(figsize=(16,8))

# plot the graph


# adjust the  y-ticks


# adjust the x-ticks and limit

# only use horizontal grid


plt.title("Total Sales by Year")

plt.show()

- By total sales per year, things are looking up!

In [ ]:
# now to the same thing for sale average


## Transactions

### Transactions count categorical columns

In [ ]:
columns = ['Segment', 'Region', 'Category']

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
sns.countplot(data=df,
              x="Segment"
             )
plt.title("Segment")
plt.xlabel('')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# now do it for every element in columns


### Daily transaction by category

In [ ]:
df_stacked = df.groupby(['Order Date', 'Category']).agg({'Order ID':'nunique'})
df_stacked = df_stacked.unstack(level='Category')
df_stacked.columns = df_stacked.columns.get_level_values(1)
df_stacked.head()

In [ ]:
# make a stacked bar of transaction for each category in the past 30 days


# And then what?

- So far the insights have been helping us to understand the data better. But does it help us to make better decision?

- So now we know a few things:
    1. Total sales by year keep rising but the average tanked.
    2. Most transactions happened for office supplies but they are among the lowest sale number (possibly due to price)

# Make it more insightful

## Transaction pattern

In [ ]:
df_s = df.groupby(["Order Date"]).agg({"Order ID": "nunique"})

df_s.rename(columns={'Order ID':'Count Order'}, inplace=True)

print(df_s.shape)
df_s.head()

In [ ]:
df_s = df_s.asfreq('D')
df_s.head()

In [ ]:
df_s.fillna(0, inplace=True)
df_s.head(10)

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))

df_s['Count Order'].plot(linewidth=1)

plt.show()

- OK this graph is not helping anyone.

- But we do see some peaks and oscilations. Let's take a look a bit more

### Daily transaction by year

In [ ]:
fig, ax = plt.subplots(4,figsize=(16,10))

ax[0].plot(df_s.loc['2015', 'Count Order'], linewidth=1)
ax[1].plot(df_s.loc['2016', 'Count Order'], linewidth=1)
ax[2].plot(df_s.loc['2017', 'Count Order'], linewidth=1)
ax[3].plot(df_s.loc['2018', 'Count Order'], linewidth=1)

plt.show()

- Now let's zoom in a bit

In [ ]:
# Plot the data for only one month for each year


- Can wee see any seasonality?

In [ ]:
# Plot the data for only one one for each year BUT with mondays as ticks


- There doesn't seem to be any seasonality pattern. The customer purchasing behavior might be a bit random (not confined by specific days).

- But let's take a look the the monthly pattern

In [ ]:
df_s['Year'] = df_s.index.year
df_s['Month'] = df_s.index.month
df_s['Weekday Name'] = df_s.index.day_name()

In [ ]:
df_s.head()

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))

sns.boxplot(data=df_s, x='Month', y='Count Order')

# Get rid of the outliers

plt.show()

- Now there's something interesting. The medians on year-ends are higher than the other months

- Can we prove it through movements?

### Transactions rolling average

In [ ]:
df_s_7d = df_s.rolling(7, center=True).mean()
df_s_7d.head(10)

In [ ]:
# get 365 days rolling average


In [ ]:
# Plot them.


- The transactions 365-day rolling average now shows a better visual on just how much the business grows. It may not be astronomical, but it kept on going up by transaction number.


- Now, remember the sum and average sales conundrum? We can use the same rolling average method to understand the real story

In [ ]:
df_s = df.groupby(["Order Date"]).agg({"Order ID": "nunique", "Sales":"sum"})

df_s.rename(columns={'Order ID':'Count Order'}, inplace=True)

print(df_s.shape)
df_s.head()

In [ ]:
df_s_7d = df_s.rolling(30, center=True).mean()
df_s_365d = df_s.rolling(window=365, center=True, min_periods=360).mean()

In [ ]:
start, end = '2015-01', '2018-12'


fig, ax = plt.subplots(figsize=(16,8))

ax.plot(df_s.loc[:, 'Sales'], marker='.', markersize=2, color='0.6', linestyle='None', label='Daily', alpha=0.3)


# Add 7-day day average and 365-day average



# Make the plot pretty.


plt.show()

- Now we understand that if we average the sale number for each year, the plot might not give us a good story because the context is simply hidden.


- The business might had some bad sale days and good sale days. Aggregating it fully would be unwise. It's better to aggregate it by neighboring context. Now we see that even if the business may not be doing well in some days, the pattern shows that it still grew by the day.

In [ ]:
# Bonus challenge: Do one per category or per region

# Customer Mapping

- In here we will try to understand the customer a little bit better.


- We will attempt to map customers based on their count of transaction and how much money they spent.


- The idea is that we can segment these customers into different quadrants and determine our power customers.

In [ ]:
df_cust = df.groupby("Customer ID").agg({'Order ID':'count',
                              'Sales': 'sum'
                              })


df_cust.rename(columns={'Order ID':'Count'}, inplace=True)

print(df_cust.shape)
df_cust.head()

- First, we check the distribution of the features

In [ ]:
sns.displot(data=df_cust,
           x='Count',
            kde=True,
            height=8,
            aspect=1.5
           )

plt.axvline(df_cust['Count'].mean(), color='r', linestyle='--')
plt.axvline(np.median(df_cust['Count']), color='g', linestyle='--')


plt.show()

In [ ]:
sns.displot(data=df_cust,
           x='Sales',
            kde=True,
            height=8,
            aspect=1.5
           )

plt.axvline(df_cust['Sales'].mean(), color='r', linestyle='--')
plt.axvline(np.median(df_cust['Sales']), color='g', linestyle='--')


plt.show()

- Now we try to map the quadrants

In [ ]:
sns.relplot(data=df_cust,
           x= 'Count',
            y = 'Sales',
            s=150,
            alpha=0.6,
            height=8, aspect=1,
            facet_kws={'size':8}
           )


# Add median with red color




plt.title("Customer Mapping")

plt.show()

- From here, we see that many customers are already considered as power customers (upper right: high transaction count and high sales total)


- There are a lot of users in the 3rd quadrant (bottom left: low sales, low transactions) which might be a segment to focus on. We can help to identify these users so that the marketing strategy can be targeted efficiently to move them to other quadrants.

# Retention


- User maps are cool and all. But a business usually relies on existing customers and make sure that they come back for more.


- There are many ways to look into retention. some of them are:
    1. L30 analysis: looking into how many times a customer purchase something in the past 30 days
    2. Cohort analysis: looking into how many times a customer purchase something since their first purchase

## L30 Analysis

In [ ]:
df_r = df.copy()
print(df_r.shape)
df_r = df_r.drop_duplicates(subset=['Order ID', 'Order Date', 'Customer ID'])
df_r = df_r[['Order Date', 'Order ID', 'Customer ID']]
print(df_r.shape)
df_r.head()

In [ ]:
df_r = df_r.groupby(['Order Date', 'Customer ID']).agg({'Order ID':'count'})
# df_r = df_r.asfreq('D')
df_r.reset_index(level= 'Customer ID', inplace=True)
df_r.head()

In [ ]:
df_r = df_r.groupby('Customer ID').rolling(30, center=False, min_periods=1).sum()
df_r.reset_index(level= 'Customer ID', inplace=True)
df_r.head()

In [ ]:
df_r = df_r.groupby(['Order Date', 'Order ID']).agg({'Order ID':'count'})
df_r.rename(columns={'Order ID':'Count'}, inplace=True)
df_r.reset_index(level='Order ID', inplace=True)
df_r.rename(columns={'Order ID':'Order Num L30'}, inplace=True)
df_r.head()

In [ ]:
df_r.loc['2018-12-30']

In [ ]:
sns.catplot(data=df_r,
           x='Order Num L30',
            y='Count',
            kind='boxen',
            height=10,
            aspect=1.5
           )

plt.show()

In [ ]:
sns.relplot(data=df_r.reset_index(),
             x='Order Date',
             y='Count',
            kind='scatter',
            s=300,
             row='Order Num L30',
            height=8,
            aspect=2,
            facet_kws={'sharex':False}
           )

plt.show()

- It seems like over the years, users are more likely to purchase more. This could be an indication that the business looks healthy. Meaning that users are willing to purchase more times in the past 30 days than when the store first open.


- However, it also indicates that the business struggles to get new customers. This is an early assumption. Can we prove it?

## Cohort

In [ ]:
df_cohort = df.groupby('Customer ID').agg({'Order Date': 'min'})
df_cohort.reset_index(inplace=True)

df_cohort['Cohort Year'] = pd.DatetimeIndex(df_cohort['Order Date']).year

df_cohort = df_cohort[['Customer ID', 'Cohort Year']]

df_cohort.head()

- First, let's take a look at how many new customers the business gathered over the years.

In [ ]:
# Make bar plot

- Yikes. Not good. Most customers came from 2015. The graph above proves that the business DOES struggle to get new customer. We need to call the marketing team.


- Let's take a look on the cohort analysis to see how many people came back from each cohort.

In [ ]:
dfx = pd.merge(df, df_cohort, on='Customer ID')
dfx['Year'] = pd.DatetimeIndex(dfx['Order Date']).year

dfx = dfx[['Cohort Year', 'Year', 'Customer ID']]

dfx.head()

In [ ]:
dfx = dfx.groupby(['Cohort Year', 'Customer ID']).agg({'Customer ID':'count'})
dfx.rename(columns={'Customer ID':'Count Trx'}, inplace=True)

In [ ]:
dfx.reset_index(inplace=True)
dfx

In [ ]:
dfx = dfx.groupby(['Cohort Year', 'Count Trx']).agg({'Customer ID':'nunique'})
dfx.rename(columns={'Customer ID': 'Unique Customer'}, inplace=True)

In [ ]:
dfx = dfx.unstack().fillna(0)
dfx

In [ ]:
dfx.columns = dfx.columns.get_level_values(1)
dfx

In [ ]:
dfx.loc[:, :20]

In [ ]:
# Make heatmap from the DataFrame

- This map tells a lot. It's telling us that customers who first purchased in 2015 are high transaction users. That's good.


- But also, since we struggled with acquiring new customers, those who are from 2016 & 2017 are not looking good. There are rooms from improvement.


- This is bascially saying that we highly depend on customers from 2015...

# Bonus: Heatmap

In [ ]:
df_h = df.groupby(['Order Date']).agg({'Order ID':'nunique',
                                        'Sales':'sum'})
df_h.rename(columns={'Order ID':'Count'}, inplace=True)

df_h.head()

In [ ]:
df_h = df_h.asfreq('D')
df_h.fillna(0, inplace=True)

df_h['Year'] = df_h.index.year
df_h['Month'] = df_h.index.month
df_h['Weekday Name'] = df_h.index.day_name()

df_h.head()

In [ ]:
df_pivot = df_h.pivot_table(index='Year', columns='Month', values='Count', aggfunc='sum')
df_pivot

## Transaction count

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))

sns.heatmap(df_pivot, 
            annot = True,
            cmap=sns.cubehelix_palette(), 
            linewidths=0.3)

ax.set_xticklabels(["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"])
ax.xaxis.tick_top()

plt.yticks(rotation=0)

plt.xlabel("")
plt.ylabel("")

plt.title("Transaction Count by Month-Year")

plt.show()

## Total sales

In [ ]:
df_pivot = df_h.pivot_table(index='Year', columns='Month', values='Sales', aggfunc='sum')
df_pivot

In [ ]:
# Make the same heatmap but for sales


In [ ]:
# Make the same heatmap but for shipping count


- From these charts, we have a better comparative density of activities (transactions, sales, and shipping activity).



- It does seem like the business have to plan ahead for year-end since the traffic would be far more than usual.